In [43]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.models import Sequential
from PIL import Image
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from IPython.display import display

x_train = pd.read_csv("data/fashion-mnist_train.csv")
x_test = pd.read_csv("data/fashion-mnist_test.csv")

y_train_labels = x_train['label']
y_test_labels = x_test['label']

x_train.drop(["label"], axis=1, inplace=True)
x_test.drop(["label"], axis=1, inplace=True)


img_array = np.array(x_train.iloc[3].values, dtype=np.uint8)
img_reshape = img_array.reshape((28,28))
image = Image.fromarray(img_reshape)
display(image)

## Data preparation

In [44]:
# Label encoding
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train_labels.to_frame())
y_test = encoder.fit_transform(y_test_labels.to_frame())

# Data normalization
x_train = (x_train / 255) - 0.5
x_test  = (x_test / 255) - 0.5

'''
28, 28, 1 specifies the dimensions of each image: height, width, and channels (1 for grayscale).

-1 in the reshape() function automatically calculates the batch size based on the total number of samples in x_train
'''

# Reshape the image
x_train = x_train.values.reshape(-1, 28, 28, 1) 
x_test = x_test.values.reshape(-1, 28, 28, 1) 

# x_test = np.expand_dims(x_test, axis=-3)

## Train

In [45]:
conv_num_filters = 8
conv_filter_size = 3
pool_size = 2

print(x_train.shape)
model = Sequential([
    Conv2D(conv_num_filters,conv_filter_size),
    MaxPooling2D(pool_size=pool_size),
    Flatten(),
    Dense(10, activation='softmax') # 10 neurons = 10 classes
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(
    x_train,
    y_train,
    epochs=3,
    validation_data=(x_test,y_test),
    
)

(60000, 28, 28, 1)
Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.7711 - loss: 0.6549 - val_accuracy: 0.8649 - val_loss: 0.3955
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8669 - loss: 0.3824 - val_accuracy: 0.8780 - val_loss: 0.3619
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.8791 - loss: 0.3480 - val_accuracy: 0.8809 - val_loss: 0.3410
